# Aggregating crime data

`crime_agg_category.csv`: 32 columns (crime counts broken down by offense **category**)

`crime_agg_name.csv`: 54 columns (crime counts broken down by offense **name**)

### Creates two aggregated datasets. Both include:
- First, ALL values in `crime_against`, `offense_name`, and `offense_category` are first renamed to shorter alternatives in *snake_case*. This was done in preparation for dummifying those columns, to make for friendly column names.
- Data is grouped by year and county, and include the following aggregated columns:
  - Crime count
  - Average age
  - Mode quarter (which quarter had the most crimes?)
  - Mode month (which month had the most crimes?)
  - Mode day of week (mon-fri => 1-7)
  - Mode hour of day (military time)
  - The original `crime_against` column was dummified, and summed during aggregation, to show total crime counts for each:
    - `against_person`
    - `against_society`
    - `against_property`
    - `not_a_crime`

#### Additional columns in `crime_agg_category` data:
- The original `offense_category` column was dummified, and then summed during aggregation, showing total crime counts broken up by offense category

#### Additional columns in `crime_agg_name` data:
- The original `offense_name` column was dummified, and then summed during aggregation, showing total crime counts broken up by offense name.

In [1]:
import crime as cr
import pandas as pd, numpy as np

INDEX = ['year', 'county']
df_raw = pd.read_csv("output/all.csv")
df_raw

,year,county,date,quarter,month,day_of_week,hour,age,crime_against,offense_name,offense_category
0,1997,BOULDER,1997-03-14,1,3,4,NaN,15.0,Person,Fondling,Sex Offenses
1,1997,BOULDER,1997-07-02,3,7,2,21.0,14.0,Property,Arson,Arson
2,1997,KIT CARSON,1997-01-20,1,1,0,22.0,58.0,Person,Simple Assault,Assault Offenses
3,1997,KIT CARSON,1997-01-18,1,1,5,NaN,21.0,Property,All Other Larceny,Larceny/Theft Offenses
4,1997,KIT CARSON,1997-03-31,1,3,0,NaN,NaN,Property,Destruction/Damage/Vandalism of Property,Destruction/Damage/Vandalism of Property
...,...,...,...,...,...,...,...,...,...,...,...
6770661,2019,BOULDER,2019-10-16,4,10,2,21.0,NaN,Property,All Other Larceny,Larceny/Theft Offenses
6770662,2019,BOULDER,2019-10-16,4,10,2,21.0,NaN,Property,All Other Larceny,Larceny/Theft Offenses
6770663,2019,ARAPAHOE,2019-06-01,2,6,5,18.0,20.0,Property,Shoplifting,Larceny/Theft Offenses
6770664,2019,ADAMS,2019-01-21,1,1,0,12.0,15.0,Property,Destruction/Damage/Vandalism of Property,Destruction/Damage/Vandalism of Property


#### Remap all values in categorical columns based on excel sheet
- We created 3 tables by hand in excel to rename EACH value in offense_name, offense_category and crime_against. This needed to be done in order to create dummy columns with friendly names.

In [2]:
xl = pd.ExcelFile("crime_renaming_map.xlsx")
name = pd.read_excel(xl, sheet_name='offense_name')
cat = pd.read_excel(xl, sheet_name='offense_category')
against = pd.read_excel(xl, sheet_name='crime_against')

df = df_raw.copy()
# Create dict from 2 cols from excel file, and pass it to series.map()
df.offense_name = df.offense_name.map(dict(zip(name.OLD, name.NEW)))
df.offense_category = df.offense_category.map(dict(zip(cat.OLD, cat.NEW)))
df.crime_against = df.crime_against.map(dict(zip(against.OLD, against.NEW)))
df_refactored = df
display(name, cat, against, df_refactored)

,OLD,NEW
0,Simple Assault,assault_simple
1,Intimidation,intimidation
2,Fondling,fondling
3,Rape,rape
4,Impersonation,impersonation
5,Robbery,robbery
6,Arson,arson
7,Destruction/Damage/Vandalism of Property,property_damage
8,Theft From Motor Vehicle,theft_from_vehicle
9,Burglary/Breaking & Entering,burglary


,OLD,NEW
0,Assault Offenses,assault
1,Sex Offenses,sex_offense
2,Fraud Offenses,fraud
3,Robbery,robbery
4,Arson,arson
5,Destruction/Damage/Vandalism of Property,property_damage
6,Larceny/Theft Offenses,larceny_theft
7,Burglary/Breaking & Entering,burglary
8,Homicide Offenses,homicide
9,Drug/Narcotic Offenses,drug


,OLD,NEW
0,Person,against_person
1,Property,against_property
2,Society,against_society
3,Not a Crime,not_a_crime


,year,county,date,quarter,month,day_of_week,hour,age,crime_against,offense_name,offense_category
0,1997,BOULDER,1997-03-14,1,3,4,NaN,15.0,against_person,fondling,sex_offense
1,1997,BOULDER,1997-07-02,3,7,2,21.0,14.0,against_property,arson,arson
2,1997,KIT CARSON,1997-01-20,1,1,0,22.0,58.0,against_person,assault_simple,assault
3,1997,KIT CARSON,1997-01-18,1,1,5,NaN,21.0,against_property,other_larceny,larceny_theft
4,1997,KIT CARSON,1997-03-31,1,3,0,NaN,NaN,against_property,property_damage,property_damage
...,...,...,...,...,...,...,...,...,...,...,...
6770661,2019,BOULDER,2019-10-16,4,10,2,21.0,NaN,against_property,other_larceny,larceny_theft
6770662,2019,BOULDER,2019-10-16,4,10,2,21.0,NaN,against_property,other_larceny,larceny_theft
6770663,2019,ARAPAHOE,2019-06-01,2,6,5,18.0,20.0,against_property,shoplifting,larceny_theft
6770664,2019,ADAMS,2019-01-21,1,1,0,12.0,15.0,against_property,property_damage,property_damage


### Aggregated datasets
1. Version 1: includes crime_category dummy sums
2. Version 2: includes crime_name dummy sums

In [3]:
df = df_refactored.copy()

def dummies_special(df, include, exclude) -> pd.DataFrame:
    return pd.get_dummies(df,
            columns=['crime_against', include],
            prefix="", prefix_sep=""
        ).drop(
            columns=[exclude, 'date', 'quarter', 'month', 'day_of_week', 'hour', 'age']
        ).groupby(INDEX).sum().reset_index()

dum_cat = dummies_special(df, 'offense_category', 'offense_name')
dum_name = dummies_special(df, 'offense_name', 'offense_category')

In [4]:
# Convert these to modes
df_modes = df[INDEX + ['quarter', 'month', 'day_of_week', 'hour']]

# For the record, pandas.Series.mode() totally sucks!!! When there's multiple modes
# it puts each of them in a numpy.ndarray as a VALUE in the cell, so you have mixed values.
# And you can't even safely index it because sometimes those arrays are EMPTY :(
# Pandas, for the love of god please give us the option to return only one mode.
# I applied the following function to fix this.
def first_in_list(x):
    """ pd.Series.mode returns ndarray when multiple modes. Safely convert to float """
    if type(x) == np.ndarray:
        if x.size > 0:
            return float(x[0])
        return np.nan
    return float(x)

# df.applymap() is just like apply but instead of acting on an axis, it acts on each cell in df
df_modes = df_modes.groupby(INDEX).agg(pd.Series.mode).applymap(first_in_list).reset_index()

# Append '_mode' to the end of each col name
for c in df_modes.columns:
    if c not in INDEX:
        df_modes = df_modes.rename(columns={c: f'{c}_mode'})

In [5]:
# Convert count and average
df_count = df[INDEX + ['date']].groupby(INDEX).count().reset_index().rename(columns={'date': 'count'})

df_avg = df[INDEX + ['age']].groupby(INDEX).mean().reset_index().rename(columns={'age': 'age_avg'})

#### Stitching everything together

In [6]:
# Numerical aggregations: counts, avgs, modes
df = df_count.merge(df_avg, on=INDEX)
df = df.merge(df_modes, on=INDEX)

In [7]:
# Summed aggregations for dummies: 2 versions
df_cat = df.merge(dum_cat, on=INDEX)
df_name = df.merge(dum_name, on=INDEX)

### Output

In [8]:
df_cat.to_csv("output/crime_agg_category.csv", index=False)
df_name.to_csv("output/crime_agg_name.csv", index=False)

In [9]:
df_cat.head()

,year,county,count,age_avg,quarter_mode,month_mode,day_of_week_mode,hour_mode,against_person,against_property,...,kidnapping,larceny_theft,porn,property_damage,prostitution,robbery,sex_offense,stolen_property,vehicle_theft,weapon_law
0,1997,ADAMS,22947,24.582071,1.0,3.0,0.0,17.0,3047.0,17766.0,...,50.0,8023.0,1.0,5467.0,14.0,189.0,316.0,245.0,1317.0,274.0
1,1997,ALAMOSA,404,27.098901,3.0,8.0,5.0,18.0,101.0,264.0,...,0.0,165.0,0.0,73.0,0.0,0.0,9.0,0.0,6.0,4.0
2,1997,ARAPAHOE,37555,25.209156,3.0,8.0,4.0,18.0,4568.0,28573.0,...,249.0,14345.0,0.0,5856.0,198.0,238.0,434.0,421.0,2537.0,695.0
3,1997,ARCHULETA,578,23.603053,4.0,10.0,4.0,9.0,24.0,487.0,...,1.0,291.0,0.0,129.0,0.0,0.0,6.0,0.0,10.0,10.0
4,1997,BACA,49,22.461538,1.0,1.0,2.0,11.0,1.0,48.0,...,0.0,19.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_name.head()

,year,county,count,age_avg,quarter_mode,month_mode,day_of_week_mode,hour_mode,against_person,against_property,...,shoplifting,sodomy,stolen_property,theft_from_building,theft_from_vehicle,theft_from_vending_machine,vehicle_part_theft,vehicle_theft,weapon_law,wire_fraud
0,1997,ADAMS,22947,24.582071,1.0,3.0,0.0,17.0,3047.0,17766.0,...,1323.0,3.0,245.0,1232.0,1733.0,53.0,1293.0,1317.0,274.0,0.0
1,1997,ALAMOSA,404,27.098901,3.0,8.0,5.0,18.0,101.0,264.0,...,0.0,0.0,0.0,14.0,7.0,10.0,1.0,6.0,4.0,1.0
2,1997,ARAPAHOE,37555,25.209156,3.0,8.0,4.0,18.0,4568.0,28573.0,...,4134.0,0.0,421.0,2283.0,3867.0,72.0,677.0,2537.0,695.0,2.0
3,1997,ARCHULETA,578,23.603053,4.0,10.0,4.0,9.0,24.0,487.0,...,30.0,0.0,0.0,31.0,58.0,0.0,3.0,10.0,10.0,0.0
4,1997,BACA,49,22.461538,1.0,1.0,2.0,11.0,1.0,48.0,...,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0
